<a href="https://colab.research.google.com/github/ria-heikkila/animal_classifier/blob/main/animal_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras import regularizers, optimizers
from keras.optimizers import Adam
from keras.models import Model, Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
import numpy as np

In [2]:
image_size = 220
classes = ["otter", "raccoon", "seal"]
num_classes = len(classes)

In [3]:
!unzip data_for_handson2.zip

Archive:  data_for_handson2.zip
  inflating: data_for_handson2/train/otter/10005174606.jpg  
  inflating: data_for_handson2/train/otter/10135070355.jpg  
  inflating: data_for_handson2/train/otter/1031153047.jpg  
  inflating: data_for_handson2/train/otter/10510544315.jpg  
  inflating: data_for_handson2/train/otter/10692642136.jpg  
  inflating: data_for_handson2/train/otter/11339008345.jpg  
  inflating: data_for_handson2/train/otter/11545180954.jpg  
  inflating: data_for_handson2/train/otter/11684891843.jpg  
  inflating: data_for_handson2/train/otter/12066677345.jpg  
  inflating: data_for_handson2/train/otter/12129403143.jpg  
  inflating: data_for_handson2/train/otter/12543081465.jpg  
  inflating: data_for_handson2/train/otter/12822903605.jpg  
  inflating: data_for_handson2/train/otter/12824087654.jpg  
  inflating: data_for_handson2/train/otter/13221828214.jpg  
  inflating: data_for_handson2/train/otter/13967460238.jpg  
  inflating: data_for_handson2/train/otter/14069841607

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "data_for_handson2/train"
validate_dir = "data_for_handson2/validate"
batch_size = 16

#Training Set
train_set = train_datagen.flow_from_directory(train_dir,
                                             target_size=(image_size,image_size),
                                             batch_size=batch_size,
                                             class_mode='categorical')
#Validation Set
test_set = test_datagen.flow_from_directory(validate_dir,
                                           target_size=(image_size,image_size),
                                           batch_size =batch_size,
                                           class_mode='categorical',
                                           shuffle=False)

Found 700 images belonging to 3 classes.
Found 250 images belonging to 3 classes.


In [5]:
#モデル定義
model = Sequential()
#畳み込み処理では画像の特徴を出すためにフィルターをかける
#padding='same'とは、畳み込み結果が同じサイズにする
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(image_size, image_size, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
#プーリング処理では画像の特徴を粗く整理する
model.add(MaxPooling2D(pool_size=(2,2))) #Max Poolingは各範囲で最大値を選択して圧縮する
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [7]:

#モデルのコンパイル
model.compile(
    optimizer = Adam(),
    loss = 'categorical_crossentropy',
    metrics = ["accuracy"]
)

In [8]:
hist = model.fit_generator(
    train_set,
    steps_per_epoch=700 // batch_size,
    epochs=20,
    validation_data=test_set,
    validation_steps=250 // batch_size
)
model.save('animal_model.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
43/43 [==============================] - 9s 218ms/step - loss: 1.8874 - accuracy: 0.4152 - val_loss: 0.9982 - val_accuracy: 0.6125
Epoch 2/20
43/43 [==============================] - 9s 218ms/step - loss: 1.0524 - accuracy: 0.4738 - val_loss: 0.9461 - val_accuracy: 0.4667
Epoch 3/20
43/43 [==============================] - 9s 220ms/step - loss: 0.9595 - accuracy: 0.5307 - val_loss: 0.8757 - val_accuracy: 0.5833
Epoch 4/20
43/43 [==============================] - 9s 219ms/step - loss: 0.9306 - accuracy: 0.5556 - val_loss: 0.8564 - val_accuracy: 0.6125
Epoch 5/20
43/43 [==============================] - 9s 217ms/step - loss: 0.9004 - accuracy: 0.5833 - val_loss: 0.8593 - val_accuracy: 0.6042
Epoch 6/20
43/43 [==============================] - 9s 218ms/step - loss: 0.9165 - accuracy: 0.5614 - val_loss: 0.8782 - val_accuracy: 0.6250
Epoch 7/20
43/43 [==============================] - 9s 217ms/step - loss

In [9]:
test_loss, test_acc = model.evaluate_generator(test_set, steps=250)
print('test loss:', test_loss)
print('test acc:', test_acc)

Instructions for updating:
Please use Model.evaluate, which supports generators.
test loss: 0.8230910897254944
test acc: 0.6079999804496765
